### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.sales-store"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
sales_store = df_processados['sales_store']

print(f"Colunas: {sales_store.shape[1]}\nLinhas: {sales_store.shape[0]}")

In [ ]:
# Identificar duplicadas com base em 'businessentityid'
duplicadas = sales_store[
    sales_store.duplicated(subset=['businessentityid'], keep=False)
]

# Verificar se existem duplicadas
if not duplicadas.empty:
    # Ordenar duplicadas por 'businessentityid' e 'modifieddate'
    duplicadas_ordenadas = duplicadas.sort_values(by=['businessentityid', 'modifieddate'])

    # Exibir duplicadas ordenadas
    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

In [ ]:
# Remover duplicadas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

sales_store = sales_store.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(sales_store)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = sales_store.copy()


#doc: bkp = backup (cópia)

In [ ]:
# Ordenar e exibir o df por 'businessentityid'
sales_store = sales_store.sort_values(by=['businessentityid'])

print(sales_store)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in sales_store.columns:   
    missing_rows = sales_store[sales_store[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
#deletar coluna desnecessárias
sales_store = sales_store.drop(columns=['demographics'])

print("Colunas restantes após remoção:")
print(sales_store.columns)

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = sales_store.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

#doc: currentflag possue somente 1 valor, o que indica que pode ser somente valores True ou False.

In [ ]:
#verificar informações do df
sales_store.info()

In [ ]:
#avaliando as variáveis qualitativas*

coluna_quantitativa = ["name"]
for col in coluna_quantitativa:
    counts = sales_store[col].value_counts().nlargest(10)
    percentages = (counts / sales_store.shape[0] * 100).map("{:.2f}%".format)
    summary = pd.DataFrame({"qtde.": counts, "%": percentages})
    print(summary)    


#doc*: variáveis qualitativas são um tipo de variável estatística que representam características ou atributos dos dados, sem serem medidas numericamente
#      no nosso caso, name, por exemplo

In [ ]:
# Dados ajustados com as 5 primeiras lojas, em ordem decrescente
top_5_stores = sales_store['name'].value_counts().nlargest(5).sort_values(ascending=True)

names = top_5_stores.index
values = top_5_stores.values

# Criar o gráfico de barras horizontais
plt.figure(figsize=(6, 3))
plt.barh(names, values, color='green', edgecolor='black', alpha=0.7)

# Adicionar título
plt.title('Lojas vs Quantidade', fontsize=14, color='white')

# Ajustar cor do plano de fundo e rótulos
plt.gca().set_facecolor('black')
plt.gcf().set_facecolor('black')
plt.tick_params(axis='x', bottom=False, labelbottom=False)  # Remove valores do eixo X
plt.tick_params(axis='y', colors='white')

# Remover títulos dos eixos
plt.xlabel("")
plt.ylabel("")

# Adicionar valores ao lado das barras
for index, value in enumerate(values):
    plt.text(value + 0.1, index, f'{value}', va='center', fontsize=12, color='white')

# Mostrar o gráfico
plt.tight_layout()
plt.show()


In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['sales_store'] = sales_store

In [ ]:
sales_store.head()

In [ ]:
# Padronizar colunas com valores textuais
sales_store['name'] = sales_store['name'].str.strip().str.upper()
sales_store['rowguid'] = sales_store['rowguid'].str.strip().str.upper()

print(sales_store.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:

# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Nome da tabela no BigQuery
    output_table = f"{output_dataset}.{table_name}"

    # Configurar job de exportação
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    # Exportar DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")